In [2]:
import pandas as pd
import numpy as np
import scipy.stats as scs


def teste_chi2(df, campo_analise):
    dados = df[['internet_via_celular', campo_analise]].dropna()
    categorias = list(dados[campo_analise].unique()) # Vê a quantidade de valores distintos no campo
    n_categorias = len(categorias)

    # Faz a contagem de pessoas com internet e sem internet dependendo de sua categoria
    tem_internet = dados[dados['internet_via_celular'] == 1]
    nao_tem_internet = dados[dados['internet_via_celular'] == 0]
    tabela_tem_internet = []
    tabela_nao_tem_internet = []
    total_categoria = []
    for categoria in categorias:
        tabela_tem_internet.append(len(tem_internet[tem_internet[campo_analise] == categoria]))
        tabela_nao_tem_internet.append(len(nao_tem_internet[nao_tem_internet[campo_analise] == categoria]))

    count = 0
    for i in tabela_tem_internet:
        count = count + i
    tabela_tem_internet.append(count)


    count = 0
    for i in tabela_nao_tem_internet:
        count = count + i
    tabela_nao_tem_internet.append(count)

    for i in range(0, n_categorias+1):
        total_categoria.append(tabela_tem_internet[i] + tabela_nao_tem_internet[i])

    tabela = np.array([tabela_tem_internet, tabela_nao_tem_internet, total_categoria])

    # Cria matriz de probabilidade
    probabilidades = np.zeros((3,n_categorias+1))
    for i in range(0, n_categorias+1):
        probabilidades[0][i] = tabela[0][i]/tabela[2][i]
        probabilidades[1][i] = tabela[1][i]/tabela[2][i]
        probabilidades[2][i] = 1

    # Cria matriz de valores esperados de acordo com a probabilidade global de ter ou não internet
    # de acordo com a categoria.
    esperado = np.zeros((2, n_categorias))
    for i in range(0, 2):
        for j in range(n_categorias):
            esperado[i][j] = tabela[2][j] * probabilidades[i][n_categorias]
            
    print("Analisando %s" % campo_analise)
    print("--------------")
    print()
    tabela = np.array([tabela[0][:-1], tabela[1][:-1]])
    print("Tabela para análise")
    print(categorias)
    print(tabela)
    print()
    print("Tabela esperada")
    print(categorias)
    print(np.ceil(esperado))
    erro = esperado
    for i in range(0,2):
        for j in range(0, n_categorias):
            erro[i][j] = abs(1-tabela[i][j]/esperado[i][j])
    print()
    print("Probabilidade")
    print(categorias)
    print(probabilidades)
    print()
    print("Diferença entre dados base e dados esperados")
    print(categorias)
    print(erro)

    print()
    print("Diferença média:")
    print(np.mean(erro))
    
df = pd.read_csv(r'./pnad.csv')

com_internet = df[df['internet_via_celular'] == 1]
print()
print("Percentual de pessoas com conexão à internet via celular no Brasil: %f %%" % (len(com_internet)/len(df)))
print()
campo_desejado = input("Digite o nome do campo que deseja comparar (nomes iguais ao dos campos do arquivo csv): ")
teste_chi2(df, campo_desejado)

# Formato das tabelas que serão geradas
#                        |
#  possui internet       |  valores...
#                        |
# ---------------------- | -----------------
#                        |
#  não possui internet   |  valores...
#                        |
# ---------------------- |


Percentual de pessoas com conexão à internet via celular no Brasil: 0.447725 %

Digite o nome do campo que deseja comparar (nomes iguais ao dos campos do arquivo csv): alfabetizado
Analisando alfabetizado
--------------

Tabela para análise
[1, 0]
[[159581    214]
 [144395  52714]]

Tabela esperada
[1, 0]
[[136098.  23698.]
 [167879.  29231.]]

Probabilidade
[1, 0]
[[0.52497895 0.00404323 0.44772544]
 [0.47502105 0.99595677 0.55227456]
 [1.         1.         1.        ]]

Diferença entre dados base e dados esperados
[1, 0]
[[0.17254661 0.9909694 ]
 [0.13988243 0.80337253]]

Diferença média:
0.5266927421981271
